In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


#HEre how to generate sources in 2D --> apply & add modification of source position in Hook 

import inspect
from pathlib import Path
import pde 
from pde import PDE, ScalarField, CartesianGrid,UnitGrid,PlotTracker,FieldCollection
import numpy as np
from numba import njit
import matplotlib.pyplot as plt
import pandas as pd


# In[5]:

@njit
def Brownian_dt(pos,D,dt,grid_size):
    return (pos+np.sqrt(2*D)*np.random.normal(loc=0,scale=np.sqrt(dt),size=len(pos)))%grid_size
    
def MapPos_togrid(pos,grid_spacing,grid_size):
    return np.uint(np.floor(pos/grid_spacing)%grid_size)


# In[1]:


#Define variables
#Dimer simulations
#test reaction_rate: increase to ability to change 1 voxel each sim step

grid_size = 200 #1 mum3
grid_spacing = 0.005 # each voxel is 0.005 mum3
timestep = 1e-3
D_prot = 7.679E-2 #in mum2/ms | conversion factor is 1E14
D_subst = 4.902E-1/grid_spacing 
D_prod = 4.949E-1/grid_spacing
length = 50
rct_rate = 2*137.5 #dimer, taken from kcat in ms-1
subst_c0 = 1.375E-1 # in 1 mum3: 1.1e6 particles,stress conditions: 3 mM --> 1.1E-3 particles /nm3 --> divide by 2
sysname = 'Dimer_inc_rate'
repno = 1 



In [ ]:
#####Bug in brown: adjust scale, is currently too high #fixed
#####Bug in initial set of background 
#####bug in materiostat: **2 needs 2 be adjusted

In [ ]:
grid = UnitGrid( [grid_size,grid_size,grid_size], periodic=True)
background = ScalarField.from_expression(grid,'+0') 
pos = np.array((int(grid_size/2),int(grid_size/2),int(grid_size/2)))
state = ScalarField(background.grid)
background.data[MapPos_togrid(pos,grid_spacing,grid_size)[0],MapPos_togrid(pos,grid_spacing,grid_size)[1],MapPos_togrid(pos,grid_spacing,grid_size)[2]] = 1

state = ScalarField(background.grid)  

def post_step_hook(state_data, t):
    global pos
    #simple solution: set negative mass to 0
    #s.data = np.maximum(s.data,0)
    #'Massostat'
    s.data,p.data = s.data-(((s.data<0)*s.data).sum(0)/grid_size**2),p.data+(((s.data<0)*s.data).sum(0)/grid_size**2)
    background.data[MapPos_togrid(pos,grid_spacing,grid_size)[0],MapPos_togrid(pos,grid_spacing,grid_size)[1],MapPos_togrid(pos,grid_spacing,grid_size)[2]] = 0
    pos = Brownian_dt(pos,D_prot,timestep,grid_size)
    background.data[MapPos_togrid(pos,grid_spacing,grid_size)[0],MapPos_togrid(pos,grid_spacing,grid_size)[1],MapPos_togrid(pos,grid_spacing,grid_size)[2]]= 1


storage = pde.MemoryStorage()

trackers = [
    "progress",  # show progress bar during simulation
    #"steady_state",  # abort when steady state is reached
    storage.tracker(interrupts=1),  # store data every simulation step
    pde.PlotTracker(show=True),  # show images during simulation
    # print some output every 1 real seconds:
    pde.PrintTracker(interrupts=pde.RealtimeInterrupts(duration=0.1)),
]

# define the pde
eq = PDE({"s": "D1*laplace(s) - k*source*s",
         "p": "D2*laplace(p) + k*source*s"}, 
         consts={"source": background,"D1":D_subst,"D2":D_prod,"k":rct_rate},
         post_step_hook=post_step_hook)
s,p = ScalarField(grid,label="d(Substrate)")+subst_c0,ScalarField(grid,label="d(Product)")
state = FieldCollection([s,p])

result = eq.solve(state, t_range=length, dt=timestep, scheme="runge-kutta" , tracker=trackers)
#result.plot
# In[ ]:

  0%|          | 0/50.0 [00:00<?, ?it/s]

Output()

t=0, c=0.0688±0.0687
t=0.004, c=0.0688±0.0687
t=0.005, c=0.0688±0.0687
t=0.007, c=0.0688±0.0687
t=0.008, c=0.0688±0.0687
t=0.009, c=0.0688±0.0687
t=0.01, c=0.0688±0.0687
t=0.011, c=0.0688±0.0687
t=0.012, c=0.0688±0.0687
t=0.013, c=0.0688±0.0687
t=0.014, c=0.0688±0.0687
t=0.015, c=0.0688±0.0687
t=0.016, c=0.0688±0.0687
t=0.017, c=0.0688±0.0687
t=0.018, c=0.0688±0.0687
t=0.019, c=0.0688±0.0687
t=0.02, c=0.0688±0.0687
t=0.021, c=0.0688±0.0687
t=0.022, c=0.0688±0.0687
t=0.023, c=0.0688±0.0687
t=0.024, c=0.0688±0.0687
t=0.025, c=0.0688±0.0687
t=0.026, c=0.0688±0.0687
t=0.027, c=0.0688±0.0687
t=0.028, c=0.0688±0.0687
t=0.029, c=0.0688±0.0687
t=0.03, c=0.0688±0.0687
t=0.031, c=0.0688±0.0687
t=0.032, c=0.0688±0.0687
t=0.033, c=0.0688±0.0687
t=0.034, c=0.0688±0.0687
t=0.035, c=0.0688±0.0687
t=0.036, c=0.0688±0.0687
t=0.037, c=0.0688±0.0687
t=0.038, c=0.0688±0.0687
t=0.039, c=0.0688±0.0687
t=0.04, c=0.0688±0.0687
t=0.041, c=0.0688±0.0687
t=0.042, c=0.0688±0.0687
t=0.043, c=0.0688±0.0687
t=0.044,

In [ ]:
thresholds = 10**np.float32(np.arange(-10,1,0.1))
tmp = np.empty((len(thresholds),(len(storage))))
for i,field in enumerate(storage):
    for j,k in enumerate(thresholds):
        tmp[j][i] = (field[1].data >= k).sum()/grid_size**3


# In[ ]:


result=np.empty((len(thresholds)))
for i,j in enumerate(tmp):
    result[i] = (np.argmax(j >= 0.5))



In [ ]:
result
pd.DataFrame({'Concentration': thresholds[:],
              '1/2 saturation reached at timestep': result[:]*timestep,
             'System': np.full((len(result)), sysname),
             'Replica': np.full((len(result)),repno)
                    }).to_csv(str(sysname+'.csv'))

In [ ]:
result

In [ ]:
storage.write_mode